In [0]:
import pyspark

In [0]:
df=spark.sql('select * from seeds_dataset_1_csv')

In [0]:
df.printSchema()

root
-- area: double (nullable = true)
-- perimeter: double (nullable = true)
-- compactness: double (nullable = true)
-- length_of_kernel: double (nullable = true)
-- width_of_kernel: double (nullable = true)
-- asymmetry_coefficient: double (nullable = true)
-- length_of_groove: double (nullable = true)

In [0]:
from pyspark.ml.clustering import KMeans

In [0]:
df.columns

Out[6]: ['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [0]:
from pyspark.ml.feature import VectorAssembler


In [0]:
assembler=VectorAssembler(inputCols=df.columns,outputCol='features')

In [0]:
final_data=assembler.transform(df)

In [0]:
final_data.printSchema()

root
-- area: double (nullable = true)
-- perimeter: double (nullable = true)
-- compactness: double (nullable = true)
-- length_of_kernel: double (nullable = true)
-- width_of_kernel: double (nullable = true)
-- asymmetry_coefficient: double (nullable = true)
-- length_of_groove: double (nullable = true)
-- features: vector (nullable = true)

In [0]:
from pyspark.ml.feature import StandardScaler

In [0]:
scaler=StandardScaler(inputCol='features',outputCol='Scaledfeatures')

In [0]:
scaler_model=scaler.fit(final_data)

In [0]:
final_data=scaler_model.transform(final_data)

In [0]:
final_data.head(1)

Out[18]: [Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), Scaledfeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))]

In [0]:
kmeans=KMeans(featuresCol='Scaledfeatures',k=3)

In [0]:
model=kmeans.fit(final_data)

In [0]:
from pyspark.ml.evaluation import ClusteringEvaluator

In [0]:
predictions = model.transform(final_data)
predictions1=predictions.select('features','prediction')
predictions1.show()

+--------------------+----------+
 features|prediction|
+--------------------+----------+
[15.26,14.84,0.87...| 0|
[14.88,14.57,0.88...| 0|
[14.29,14.09,0.90...| 0|
[13.84,13.94,0.89...| 0|
[16.14,14.99,0.90...| 0|
[14.38,14.21,0.89...| 0|
[14.69,14.49,0.87...| 0|
[14.11,14.1,0.891...| 0|
[16.63,15.46,0.87...| 1|
[16.44,15.25,0.88...| 0|
[15.26,14.85,0.86...| 0|
[14.03,14.16,0.87...| 0|
[13.89,14.02,0.88...| 0|
[13.78,14.06,0.87...| 0|
[13.74,14.05,0.87...| 0|
[14.59,14.28,0.89...| 0|
[13.99,13.83,0.91...| 0|
[15.69,14.75,0.90...| 0|
[14.7,14.21,0.915...| 0|
[12.72,13.57,0.86...| 2|
+--------------------+----------+
only showing top 20 rows

In [0]:
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
silhouette

Out[25]: 0.616267393520126

In [0]:
centers = model.clusterCenters()
centers

Out[26]: [array([ 4.93382436, 10.94691274, 37.30542404, 12.41332714, 8.60366812,
 1.82917353, 10.40106154]),
 array([ 6.35645488, 12.40730852, 37.41990178, 13.93860446, 9.7892399 ,
 2.41585013, 12.29286107]),
 array([ 4.06660859, 10.14191893, 35.84098009, 11.81592066, 7.52397236,
 3.1823335 , 10.39801233])]